In [1]:
# Imports
import cadquery as cq
import jupyter_cadquery as jcq
import ipywidgets as widgets
from IPython.display import display
from pathlib import Path
import re
from typing import Dict
from pydantic import BaseModel, Field, ConfigDict
import pickle

# Optional: Load environment variables if needed
from dotenv import load_dotenv
load_dotenv()

Overwriting auto display for cadquery Workplane and Shape


False

In [2]:
# Shape parameter model (Pydantic v2)
class ShapeParams(BaseModel):
    shape: str = Field(..., description="cube, cylinder, sphere, plate, or tube")
    params: Dict[str, float]
    model_config = ConfigDict(extra="forbid")

# Example regex-based parameter extractor
def extract_numbers(prompt: str):
    matches = re.findall(r"(\d+(?:\.\d+)?)(?:\s*(mm|cm|m)?)", prompt)
    numbers = []
    for value, unit in matches:
        val = float(value)
        if unit == "cm":
            val *= 10
        elif unit == "m":
            val *= 1000
        numbers.append(val)
    return numbers

# Offline intent classifier (minimal, expandable)
try:
    vectorizer, clf = pickle.load(open("intent_classifier.pkl", "rb"))
except:
    # First time: create a tiny classifier
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.ensemble import RandomForestClassifier

    prompts = [
        "Make a tube 40 outer 30 inner 100 height",
        "Cube 20x20x20",
        "Cylinder diameter 50 height 100",
        "Plate 100x50 thickness 5"
    ]
    labels = ["tube", "cube", "cylinder", "plate"]
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(prompts)
    clf = RandomForestClassifier(n_estimators=100)
    clf.fit(X, labels)
    pickle.dump((vectorizer, clf), open("intent_classifier.pkl", "wb"))

def predict_shape(prompt: str):
    X_test = vectorizer.transform([prompt])
    return clf.predict(X_test)[0]


In [3]:
# Simple CAD generators
def make_cube(length, width, height):
    return cq.Workplane("XY").box(length, width, height)

def make_cylinder(diameter, height):
    radius = diameter / 2
    return cq.Workplane("XY").circle(radius).extrude(height)

def make_sphere(diameter):
    radius = diameter / 2
    return cq.Workplane("XY").sphere(radius)

def make_plate(length, width, thickness):
    return cq.Workplane("XY").box(length, width, thickness)

def make_tube(outer, wall, height):
    inner = outer - wall
    return cq.Workplane("XY").circle(outer/2).circle(inner/2).extrude(height)


In [4]:


from math import isclose

def generate_from_prompt(prompt: str):
    # Get shape type from offline classifier
    shape_type = predict_shape(prompt)
    # Extract numbers from prompt
    numbers = extract_numbers(prompt)

    # --- CUBE ---
    if shape_type == "cube":
        x = numbers[0] if len(numbers) > 0 else 20.0
        y = numbers[1] if len(numbers) > 1 else x
        z = numbers[2] if len(numbers) > 2 else x
        return make_cube(x, y, z)

    # --- CYLINDER ---
    if shape_type == "cylinder":
        d = numbers[0] if len(numbers) > 0 else 10.0
        h = numbers[1] if len(numbers) > 1 else 20.0
        return make_cylinder(d, h)

    # --- SPHERE ---
    if shape_type == "sphere":
        d = numbers[0] if len(numbers) > 0 else 10.0
        return make_sphere(d)

    # --- PLATE ---
    if shape_type == "plate":
        x = numbers[0] if len(numbers) > 0 else 100.0
        y = numbers[1] if len(numbers) > 1 else 50.0
        t = numbers[2] if len(numbers) > 2 else 5.0
        return make_plate(x, y, t)

    # --- TUBE ---
    if shape_type == "tube" or "washer" or "pipe":
        outer = numbers[0] if len(numbers) > 0 else 40.0
        inner = numbers[1] if len(numbers) > 1 else max(outer * 0.5, 10.0)
        h = numbers[2] if len(numbers) > 2 else 20.0

        # Ensure inner < outer
        if inner >= outer or isclose(inner, outer):
            inner = max(1.0, outer - 2.0)

        return make_tube(outer, inner, h)

    # --- Unknown shape fallback ---
    raise ValueError(
        "Could not interpret prompt. Try something like: 'cylinder diameter 50 mm height 100 mm'"
    )


In [5]:
# Widgets
prompt_box = widgets.Textarea(
    value='',
    placeholder='Enter CAD prompt here, e.g., "tube outer 40 inner 30 height 100"',
    description='Prompt:',
    layout=widgets.Layout(width='80%', height='80px')
)

generate_btn = widgets.Button(description="Generate CAD", button_style='success')
download_stl_btn = widgets.Button(description="Download STL", button_style='info', disabled=True)
download_step_btn = widgets.Button(description="Download STEP", button_style='warning', disabled=True)

status_out = widgets.Output()

# Arrange download buttons side by side
buttons_box = widgets.HBox([download_stl_btn, download_step_btn])

# Display all widgets
display(prompt_box, generate_btn, buttons_box, status_out)

# Global variable for current model
current_model = None

# Generate button callback
def on_generate_clicked(b):
    global current_model
    status_out.clear_output()
    with status_out:
        prompt = prompt_box.value.strip()
        if not prompt:
            print("Enter a prompt first!")
            return
        print(f"Generating model for prompt: {prompt}")
        try:
            current_model = generate_from_prompt(prompt)  # or generate_from_prompt_default
            jcq.show(current_model)
            print("Model generated!")
            download_stl_btn.disabled = False
            download_step_btn.disabled = False
        except Exception as e:
            print("Error generating model:", e)

generate_btn.on_click(on_generate_clicked)

# Download STL callback
def on_download_stl_clicked(b):
    global current_model
    if current_model is None:
        with status_out:
            print("No model to download!")
        return
    out_path = Path.cwd() / "model.stl"
    try:
        cq.exporters.export(current_model, str(out_path))
        with status_out:
            print(f"Model exported to STL: {out_path}")
    except Exception as e:
        with status_out:
            print("Failed to export STL:", e)

download_stl_btn.on_click(on_download_stl_clicked)

# Download STEP callback
def on_download_step_clicked(b):
    global current_model
    if current_model is None:
        with status_out:
            print("No model to download!")
        return
    out_path = Path.cwd() / "model.step"
    try:
        cq.exporters.export(current_model, str(out_path), exportType='STEP')
        with status_out:
            print(f"Model exported to STEP: {out_path}")
    except Exception as e:
        with status_out:
            print("Failed to export STEP:", e)

download_step_btn.on_click(on_download_step_clicked)


Textarea(value='', description='Prompt:', layout=Layout(height='80px', width='80%'), placeholder='Enter CAD pr…

Button(button_style='success', description='Generate CAD', style=ButtonStyle())

Output()